In [28]:
!pip install git+https://github.com/huggingface/datasets
! pip install -U accelerate
! pip install -U transformers
!pip install librosa
!pip install evaluate>=0.3.0
!pip install jiwer
!pip install more-itertools

  Cloning https://github.com/huggingface/datasets to /tmp/pip-req-build-oh6r5wsw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets /tmp/pip-req-build-oh6r5wsw
  Resolved https://github.com/huggingface/datasets to commit 1bf8a46cc7b096d5c547ea3794f6a4b6c31ea762
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-

# Imports

In [30]:
import IPython.display as ipd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from evaluate import load
import pandas as pd
import datasets

# Dataset

In [11]:
train_set = datasets.load_dataset("CAiRE/ASCEND", split="train")
val_set = datasets.load_dataset("CAiRE/ASCEND", split="validation")
test_set = datasets.load_dataset("CAiRE/ASCEND", split="test")

In [13]:
next(iter(train_set))

{'id': '00000',
 'path': '/storage/hf-datasets-cache/all/datasets/16739474757983-config-parquet-and-info-CAiRE-ASCEND-5c1abf9c/downloads/extracted/f0790e45797bd654a35ecd1eb4865fa761f1cbd842b674e0defb6812ae8cffbf/waves/ses1_spk1_L2_0.560_1.560.wav',
 'audio': {'path': 'ses1_spk1_L2_0.560_1.560.wav',
  'array': array([-0.00332642, -0.00146484, -0.00396729, ..., -0.0038147 ,
          0.00048828,  0.00463867]),
  'sampling_rate': 16000},
 'transcription': '我刚刚开始record',
 'duration': 1.559999942779541,
 'language': 'mixed',
 'original_speaker_id': 1,
 'session_id': 1,
 'topic': 'persona'}

In [25]:
ipd.Audio(train_set[0]["audio"]["array"], rate=16000)

In [26]:
ipd.Audio(train_set[1]["audio"]["array"], rate=16000)

# Test sample

In [47]:
wer = load("wer")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
def generate_one(audio_sample):
  transcriptions = []
  waveform = audio_sample["array"]
  sampling_rate = audio_sample["sampling_rate"]

  input_features = processor(
      waveform, sampling_rate=sampling_rate, return_tensors="pt"
  ).input_features

  predicted_ids = model.generate(input_features)

  return processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [45]:
for i in range(20):
  print(f"Sample {i+1}")
  print("Ground truth:", train_set[i]["transcription"])
  print("Transcription:", generate_one(train_set[i]["audio"])[0])

Sample 1
Ground truth: 我刚刚开始record
Transcription: 我剛剛開始入康
Sample 2
Ground truth: 嗯hello我的名字叫徐妍
Transcription: 嗯 哈囉我的名字叫許言
Sample 3
Ground truth: 嗯初次见面nice to meet you嗯
Transcription: 嗯 出事件面那是等你住嗯
Sample 4
Ground truth: 今天呢我非常希望能够通过这个机会去跟你make friends
Transcription: 今天我非常希望能夠通過這個機會去給你Mick Friends
Sample 5
Ground truth: 嗯你知道就是
Transcription: 然后教就是
Sample 6
Ground truth: 我们平时能够遇见其他stranger的机会其实不是很多所以其实这样的一个机会我还是觉得
Transcription: 我们平时能够遇见Cast Dreamer机会其实不是很多所以这样的机会我还是觉得
Sample 7
Ground truth: 很honour的
Transcription:  All nerds.
Sample 8
Ground truth: 对然后嗯我是来自中国北方的一个小城市
Transcription: 然后我是来自中国北方的一个小城市
Sample 9
Ground truth: we have the sea shore in the city and we have a lot of
Transcription:  We have the sea shore in the city and we have a lot of them.
Sample 10
Ground truth: delicious sea food
Transcription:  Delicious seafood.
Sample 11
Ground truth: 嗯我不知道好像我不太确定你家是不是也是来自一个similar city所以
Transcription: 我不知道好像我不太確定你家是不是也是來自一個新的尺寄所以
Sample 12
Ground truth: 嗯接下来我们也可以再接下来我们可以讨论一下吃海鲜啊also sea